<a href="https://www.kaggle.com/code/trakiet/duachuot?scriptVersionId=268381521" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/usercourse-cleaned/usercourse_cleaned.csv
/kaggle/input/mooccubex-dataset/course-teacher.txt
/kaggle/input/mooccubex-dataset/course.json
/kaggle/input/mooccubex-dataset/paper.json
/kaggle/input/mooccubex-dataset/reply.json
/kaggle/input/mooccubex-dataset/concept-comment.txt
/kaggle/input/mooccubex-dataset/user-video.json
/kaggle/input/mooccubex-dataset/course-comment.txt
/kaggle/input/mooccubex-dataset/concept-problem.txt
/kaggle/input/mooccubex-dataset/user-reply.txt
/kaggle/input/mooccubex-dataset/school.json
/kaggle/input/mooccubex-dataset/translated_reply.json
/kaggle/input/mooccubex-dataset/video.json
/kaggle/input/mooccubex-dataset/concept-video.txt
/kaggle/input/mooccubex-dataset/other.json
/kaggle/input/mooccubex-dataset/course-school.txt
/kaggle/input/mooccubex-dataset/problem.json
/kaggle/input/mooccubex-dataset/user-comment.txt
/kaggle/input/mooccubex-dataset/user.json
/kaggle/input/mooccubex-dataset/exercise-problem.txt
/kaggle/input/mooccubex-dataset/concept-

In [2]:
basePath = '/kaggle/input/mooccubex-dataset/'
basePathComment = '/kaggle/input/mooccomments/'

In [3]:

import pyspark 
from pyspark.sql import SparkSession
import warnings
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col, sum, from_unixtime, hour, dayofweek, count, round, when, isnull, split, avg
from pyspark.sql.types import IntegerType
import seaborn as sns
import matplotlib.pyplot as plt 
from os import truncate

warnings.filterwarnings("ignore")

In [4]:
spark = SparkSession.builder \
    .appName("MySparkApplication") \
    .master("local[*]") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/16 04:15:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
!ls {basePath}

comment-reply.txt    course-school.txt	      translated_reply.json
concept-comment.txt  course-teacher.txt       user-comment.txt
concept.json	     exercise-problem.txt     user.json
concept-other.txt    other.json		      user-problem.json
concept-paper.txt    paper.json		      user-reply.txt
concept-problem.txt  problem.json	      user-video.json
concept-reply.json   reply.json		      user-xiaomu.json
concept-video.txt    reply-reply.txt	      video_id-ccid.txt
course-comment.txt   school.json	      video.json
course-field.json    teacher.json
course.json	     translated_comment.json


In [6]:
df_user_video = spark.read.json(basePath + '/user-video.json')
df_user_video.show(truncate=True)

+--------------------+-------+
|                 seq|user_id|
+--------------------+-------+
|[{[{190.0, 158843...|  U_112|
|[{[{506.0, 160406...|  U_150|
|[{[{4556.8, 15981...|  U_172|
|[{[{109.0, 159883...|  U_189|
|[{[{64.747, 15821...|  U_197|
|[{[{206.8, 159814...|  U_382|
|[{[{598.978027343...|  U_514|
|[{[{9.388, 160075...|  U_670|
|[{[{99.5, 1580808...|  U_783|
|[{[{454.848426, 1...|  U_796|
|[{[{231.0, 160258...|  U_817|
|[{[{567.2, 160078...|  U_981|
|[{[{595.0, 157961...| U_1071|
|[{[{9.7, 15989332...| U_1073|
|[{[{324.4, 158791...| U_1193|
|[{[{924.9, 158436...| U_1206|
|[{[{36.5, 1580867...| U_1323|
|[{[{391.5, 160091...| U_1361|
|[{[{130.4, 159765...| U_1368|
|[{[{1214.0, 16037...| U_1418|
+--------------------+-------+
only showing top 20 rows



In [7]:
df_user_video.count() 

310360

In [8]:
df_user_video.printSchema()

root
 |-- seq: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- segment: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- end_point: double (nullable = true)
 |    |    |    |    |-- local_start_time: long (nullable = true)
 |    |    |    |    |-- speed: double (nullable = true)
 |    |    |    |    |-- start_point: double (nullable = true)
 |    |    |-- video_id: string (nullable = true)
 |-- user_id: string (nullable = true)



In [9]:
from pyspark.sql.functions import explode
# Explode the outer array in 'seq' to get each video sequence as a row
df_exploded = df_user_video.select("user_id", explode("seq").alias("video_sequence"))

# Select the video_id and explode the 'segment' array within each video sequence
df_exploded = df_exploded.select(
    "user_id",
    df_exploded["video_sequence"]["video_id"].alias("video_id"),
    explode(df_exploded["video_sequence"]["segment"]).alias("segment_details")
)

# Select the individul fields from the 'segment_details' struct 
df_flattened = df_exploded.select(
    "user_id", 
    "video_id", 
    df_exploded["segment_details"]["start_point"].alias("start_point"),
    df_exploded["segment_details"]["end_point"].alias("end_point"),
    df_exploded["segment_details"]["speed"].alias("speed"),
    df_exploded["segment_details"]["local_start_time"].alias("local_start_time")
)

df_flattened.show(truncate=False)

+-------+---------+-----------+---------+-----+----------------+
|user_id|video_id |start_point|end_point|speed|local_start_time|
+-------+---------+-----------+---------+-----+----------------+
|U_112  |V_1395633|130.0      |190.0    |1.0  |1588431144      |
|U_112  |V_1395633|220.0      |250.0    |1.0  |1588431234      |
|U_112  |V_1395633|478.8      |508.8    |1.0  |1588437359      |
|U_112  |V_1395633|655.1      |692.55   |1.25 |1588437514      |
|U_112  |V_1395635|135.0      |170.0    |1.0  |1588438045      |
|U_112  |V_1395636|128.3      |180.8    |1.5  |1588438278      |
|U_112  |V_1395639|100.0      |106.25   |1.25 |1588438980      |
|U_112  |V_1395639|180.0      |186.25   |1.25 |1588439045      |
|U_112  |V_6210799|9.0        |22.25    |1.25 |1601012656      |
|U_112  |V_6210799|28.0       |271.5    |1.25 |1601012671      |
|U_112  |V_6210799|279.0      |331.5    |1.5  |1601012841      |
|U_112  |V_6210799|346.0      |356.0    |1.5  |1601012886      |
|U_112  |V_6210800|6.0   

# Calculate video watching duration

In [10]:
df_flattened = df_flattened.withColumn("duration", round(col("end_point") - col("start_point")))
df_flattened = df_flattened.select("user_id", "video_id", "duration", "speed")
df_flattened.show(truncate=False)


+-------+---------+--------+-----+
|user_id|video_id |duration|speed|
+-------+---------+--------+-----+
|U_112  |V_1395633|60.0    |1.0  |
|U_112  |V_1395633|30.0    |1.0  |
|U_112  |V_1395633|30.0    |1.0  |
|U_112  |V_1395633|37.0    |1.25 |
|U_112  |V_1395635|35.0    |1.0  |
|U_112  |V_1395636|53.0    |1.5  |
|U_112  |V_1395639|6.0     |1.25 |
|U_112  |V_1395639|6.0     |1.25 |
|U_112  |V_6210799|13.0    |1.25 |
|U_112  |V_6210799|244.0   |1.25 |
|U_112  |V_6210799|53.0    |1.5  |
|U_112  |V_6210799|10.0    |1.5  |
|U_112  |V_6210800|8.0     |1.5  |
|U_150  |V_6224262|220.0   |2.0  |
|U_172  |V_6432959|5.0     |1.0  |
|U_189  |V_6334508|105.0   |1.0  |
|U_189  |V_6334508|135.0   |1.0  |
|U_189  |V_6334516|5.0     |1.0  |
|U_189  |V_6334519|840.0   |1.0  |
|U_189  |V_6334017|317.0   |1.0  |
+-------+---------+--------+-----+
only showing top 20 rows



# Video DataFrame

In [11]:
df_video = spark.read.json(basePath + 'video.json')
df_video = df_video.withColumn(
    "video_length", 
    F.array_max("end").cast(FloatType())
)
df_video.show(truncate=True)


+--------------------+--------------------+----------------------+--------------------+------------------------------------+------------+
|                ccid|                 end|                  name|               start|                                text|video_length|
+--------------------+--------------------+----------------------+--------------------+------------------------------------+------------+
|0001603F826A3DDA9...|[4.255, 8.119, 10...|                 Video|[1.031, 7.095, 8....|[第二个就是短助记符在生成上面。有...|     509.095|
|0003DB14A14A53279...|[12.581, 17.38, 1...|                 Video|[11.799, 12.581, ...|  [大家好, 今天我们讨论一维系统辐...|     367.336|
|0004A5C6F07E369D9...|[20.83, 23.39, 26...|        家禽的消化系统|[19.2, 20.84, 23....|   [各位同学 大家好, 我是来自中国...|     492.315|
|00059EBD1371A6959...|[23.14, 194.458, ...|    1.2 计算机硬件系统|[19.372, 190.522,...| [大家好 本视频介绍计算机硬件系统...|     630.757|
|0005D1DC01B4EF8F9...|[25.08, 26.775, 2...|      什么是供应链网络|[24.33, 25.081, 2...|   [大家好, 今天我们学习的知识点,...|   

In [12]:
df_video_length = df_video.select("ccid", "video_length")
df_video_length.show()

+--------------------+------------+
|                ccid|video_length|
+--------------------+------------+
|0001603F826A3DDA9...|     509.095|
|0003DB14A14A53279...|     367.336|
|0004A5C6F07E369D9...|     492.315|
|00059EBD1371A6959...|     630.757|
|0005D1DC01B4EF8F9...|      408.75|
|0007919ED7652A779...|      506.57|
|0007C4A5229EE3CF9...|     1190.78|
|0008C8CC056F4EAC9...|     130.309|
|000948D39EA6B0A69...|      421.32|
|0009510AE8068CBA9...|      622.68|
|000A1158B61C13B79...|     1281.36|
|000A40E1D226AEE09...|     255.465|
|000A5EE0C695E8EF9...|       553.8|
|000BD50F6730291F9...|     1328.56|
|000E08952A010F3D9...|       590.0|
|000F83F0945DC9D79...|      659.68|
|0012B03A05345F229...|       17.18|
|0012D2CEBDFED7729...|      623.04|
|00137081D7B0E4219...|      530.71|
|00137797685039299...|     351.188|
+--------------------+------------+
only showing top 20 rows



In [13]:
df_video_id_ccid = spark.read.text(basePath + '/video_id-ccid.txt')
df_video_id_ccid = df_video_id_ccid.select(
    split(col('value'), "\t")[0].alias("video_id"),
    split(col('value'), "\t")[1].alias("ccid")
)

df_video_id_ccid.show(truncate=False)

+---------+--------------------------------+
|video_id |ccid                            |
+---------+--------------------------------+
|V_234845 |0000363DB5B6E0869C33DC5901307461|
|V_234876 |0000363DB5B6E0869C33DC5901307461|
|V_234907 |0000363DB5B6E0869C33DC5901307461|
|V_293392 |0000363DB5B6E0869C33DC5901307461|
|V_293445 |0000363DB5B6E0869C33DC5901307461|
|V_293494 |0000363DB5B6E0869C33DC5901307461|
|V_627142 |0000363DB5B6E0869C33DC5901307461|
|V_944346 |0000363DB5B6E0869C33DC5901307461|
|V_944377 |0000363DB5B6E0869C33DC5901307461|
|V_944408 |0000363DB5B6E0869C33DC5901307461|
|V_1386166|0000363DB5B6E0869C33DC5901307461|
|V_1479635|0000363DB5B6E0869C33DC5901307461|
|V_1494506|0000363DB5B6E0869C33DC5901307461|
|V_1494536|0000363DB5B6E0869C33DC5901307461|
|V_1494566|0000363DB5B6E0869C33DC5901307461|
|V_1494614|0000363DB5B6E0869C33DC5901307461|
|V_1494625|0000363DB5B6E0869C33DC5901307461|
|V_1494656|0000363DB5B6E0869C33DC5901307461|
|V_1570352|0000363DB5B6E0869C33DC5901307461|
|V_2070578

In [14]:
df_interaction_with_length = df_flattened.join(
    df_video_id_ccid,
    on="video_id",
    how="left"
).join(
    df_video_length,
    on="ccid",
    how="left"
)

# Dọn dẹp: chỉ giữ lại các cột cần thiết cho bước tiếp theo
df_final_interaction = df_interaction_with_length.select(
    "user_id",
    "video_id",
    "duration",
    "speed",
    "video_length"
)

# df_final_interaction = df_final_interaction.with 
# Hiển thị kết quả để kiểm tra
print("Kết quả sau khi gộp df_flattened với cả hai bảng:")
df_final_interaction.show()

Kết quả sau khi gộp df_flattened với cả hai bảng:


+----------+---------+--------+-----+------------+
|   user_id| video_id|duration|speed|video_length|
+----------+---------+--------+-----+------------+
|U_10628074|V_1358596|    65.0|  1.0|      483.76|
|U_10628074|V_1358596|   125.0|  1.0|      483.76|
|U_10628074|V_1358596|    92.0|  1.0|      483.76|
|U_10628074|V_1358596|   147.0|  1.0|      483.76|
|U_10628087|V_1358592|    15.0|  1.0|      503.04|
|U_10628087|V_1358540|     5.0|  1.0|      593.84|
|U_14136049|V_1358540|    30.0|  1.0|      593.84|
|U_14136049|V_1358540|    30.0|  1.0|      593.84|
|U_30449310|V_1358540|     8.0|  1.5|      593.84|
|U_30449310|V_1358540|     8.0|  1.5|      593.84|
|U_30449310|V_1358540|     8.0|  1.5|      593.84|
|U_30449310|V_1358540|     8.0|  1.5|      593.84|
|U_30449310|V_1358540|     8.0|  1.5|      593.84|
|U_30449310|V_1358540|     8.0|  1.5|      593.84|
|U_30449310|V_1358540|     8.0|  1.5|      593.84|
|U_10628087|V_1358562|    30.0|  1.0|      667.28|
|U_10628087|V_1358562|    35.0|

In [15]:
# Bước 1: Tổng hợp dữ liệu theo user_id và video_id
df_user_video_stats = df_final_interaction.groupBy("user_id", "video_id").agg(
    F.sum("duration").alias("total_watch_time"),
    F.round(F.avg("speed"), 2).alias("mean_speed"),
    F.first("video_length").alias("video_length") # video_length là như nhau cho mỗi nhóm
)

# Bước 2: Tính toán tỷ lệ phần trăm hoàn thành
df_with_completion = df_user_video_stats.withColumn(
    "completion_percent",
    # Kiểm tra video_length > 0 để tránh lỗi chia cho 0
    F.when(
        (F.col("video_length").isNotNull()) & (F.col("video_length") > 0),
        (F.col("total_watch_time") / F.col("video_length")) * 100
    ).otherwise(0) # Nếu không có thông tin độ dài, coi như 0%
)

# Bước 3: Giới hạn % hoàn thành ở mức 100% và làm tròn
df_with_completion = df_with_completion.withColumn(
    "completion_percent",
    F.when(
        F.col("completion_percent") > 100, 100
    ).otherwise(F.col("completion_percent"))
)

df_user_video_final = df_with_completion.withColumn(
    "completion_percent", 
    F.round("completion_percent", 3)
)


In [16]:
# Quy tắc phân loại mức độ tương tác
df_user_video_final = df_user_video_final.withColumn(
    "engagement_level",
    F.when((F.col("completion_percent") > 80) & (F.col("mean_speed") <= 1.25), "Rất cao")
     .when(F.col("completion_percent") > 60, "Cao")
     .when(F.col("completion_percent") > 30, "Trung bình")
     .when(F.col("completion_percent") > 10, "Thấp")
     .otherwise("Không tương tác")
)


In [17]:
# Hiển thị kết quả cuối cùng
print("Bảng tổng hợp cuối cùng với % thời lượng đã xem:")
df_user_video_final.show()

Bảng tổng hợp cuối cùng với % thời lượng đã xem:


+----------+---------+----------------+----------+------------+------------------+----------------+
|   user_id| video_id|total_watch_time|mean_speed|video_length|completion_percent|engagement_level|
+----------+---------+----------------+----------+------------+------------------+----------------+
| U_7897406|V_1366665|           604.0|       1.0|        NULL|               0.0| Không tương tác|
|U_30308468|V_5206526|           513.0|       2.0|       494.5|             100.0|             Cao|
|U_35309346|V_6262217|           419.0|       1.5|        NULL|               0.0| Không tương tác|
|U_24634387| V_675221|             5.0|       1.0|     288.277|             1.734| Không tương tác|
|U_31409687|V_7238564|           111.0|      1.25|        NULL|               0.0| Không tương tác|
|U_31409714|V_7238564|            65.0|       1.0|        NULL|               0.0| Không tương tác|
|U_33811847|V_7294397|            25.0|       1.0|        NULL|               0.0| Không tương tác|


In [18]:
df_course = spark.read.json(basePath + '/course.json')
df_course.show(truncate=True)

+-------------------------------------+------------------------------+--------+--------------------------------+-------------+--------------------+
|                                about|                         field|      id|                            name|prerequisites|            resource|
+-------------------------------------+------------------------------+--------+--------------------------------+-------------+--------------------+
|通过老师导读，同学们可深入这一经典...|        [历史学, 中国语言文学]|C_584313|                《资治通鉴》导读|             |[{1.1.1, V_849, [...|
|本课程是理工科的一门数学基础课，系...|[应用经济学, 数学, 物理学, ...|C_584329|      微积分——极限理论与一元函数|             |[{1.1.1, V_1350, ...|
|掌握基本的摄影技能，了解图片新闻的...|          [艺术学, 新闻传播学]|C_584381|                        新闻摄影|             |[{1.1.1, V_1800, ...|
|  最有趣的理论+最有用的算法=不得不...|            [计算机科学与技术]|C_597208|            数据挖掘：理论与算法|             |[{1.1.1, V_2961, ...|
|大学计算机课程将以计算思维为导向，...|                            []|C_597225|                      大学计算机|         

In [19]:
df_user = spark.read.json(basePath + '/user.json').show(truncate=True)

+--------------------+--------------------+------+-----+------+--------+-------------+
|        course_order|         enroll_time|gender|   id|  name|  school|year_of_birth|
+--------------------+--------------------+------+-----+------+--------+-------------+
|   [682129, 2294668]|[2019-10-12 10:28...|     0| U_22|    我|        |         2015|
|[597214, 605512, ...|[2019-05-20 16:06...|     1| U_24|王帅国|清华大学|         6558|
|           [1903985]|[2020-08-07 18:59...|     0| U_25|王帅国|清华大学|         NULL|
|[696679, 1704639,...|[2020-03-01 21:24...|     1| U_53|于歆杰|清华大学|         1973|
|[682442, 682164, ...|[2019-10-09 02:17...|     2| U_54|马昱春|清华大学|         NULL|
|            [696679]|[2019-12-20 12:06...|     1| U_67|李小马|学堂在线|         6798|
|    [696692, 948431]|[2020-01-21 10:15...|     2| U_68|    秋|清华大学|         NULL|
|[375775, 375778, ...|[2019-02-26 19:21...|     1| U_69| 培源_|清华大学|         NULL|
|[676664, 707135, ...|[2020-04-22 18:23...|     0| U_90|刘俊洋| qinghua|         NULL|
|    [

In [20]:
df_reply = spark.read.json(basePath + '/reply.json')
df_reply.show()

+-------------------+-----+-------------------------------------+----------+
|        create_time|   id|                                 text|   user_id|
+-------------------+-----+-------------------------------------+----------+
|2019-08-05 12:55:54| Rp_1|                             测试回复|U_10030806|
|2019-08-09 16:39:06| Rp_2|                                   赞|U_10031397|
|2019-08-10 22:39:35| Rp_3|                               好喜欢|U_10031531|
|2019-08-12 14:43:34| Rp_4|                             你也好棒|U_10031508|
|2019-08-12 14:44:51| Rp_5|                                 嗯对|U_10031508|
|2019-08-12 14:47:58| Rp_6|                           人工智能是|U_10031508|
|2019-08-13 09:41:32| Rp_7|           我的观点就是，你说啥就时啥|U_10031536|
|2019-08-13 09:41:42| Rp_8|           我的观点就是，你说啥就时啥|U_10031536|
|2019-08-13 09:41:53| Rp_9|           我的观点就是，你说啥就时啥|U_10031536|
|2019-08-13 09:41:59|Rp_10|           我的观点就是，你说啥就时啥|U_10031536|
|2019-08-19 18:08:53|Rp_11|                                11111|U_100

In [21]:
df_user_course = spark.read.csv('/kaggle/input/usercourse-cleaned/usercourse_cleaned.csv', header=True).drop('_c0')
df_user_course.show(truncate=False)

+---------+----------+------------------------+
|course_id|user_id   |enroll_time             |
+---------+----------+------------------------+
|C_680860 |U_25140   |2019-11-18T20:14:29.000Z|
|C_676937 |U_38390   |2020-06-06T09:20:41.000Z|
|C_948409 |U_45146   |2020-02-23T18:09:35.000Z|
|C_948409 |U_126045  |2020-07-09T23:34:48.000Z|
|C_676937 |U_175157  |2019-11-02T00:07:09.000Z|
|C_883345 |U_1023243 |2020-10-07T08:47:48.000Z|
|C_676937 |U_10291924|2020-02-16T22:01:16.000Z|
|C_883345 |U_10323329|2020-09-01T19:54:48.000Z|
|C_883345 |U_10369867|2020-02-24T20:41:58.000Z|
|C_883345 |U_10552339|2020-08-22T10:25:32.000Z|
|C_883345 |U_10735076|2020-06-06T08:57:38.000Z|
|C_883345 |U_1080562 |2019-12-19T09:28:03.000Z|
|C_883345 |U_1086018 |2020-07-20T21:27:24.000Z|
|C_883345 |U_10898009|2020-03-28T17:34:57.000Z|
|C_883345 |U_10907016|2020-08-12T00:54:25.000Z|
|C_883345 |U_10953321|2020-08-01T17:59:17.000Z|
|C_676937 |U_11053311|2020-08-27T21:09:06.000Z|
|C_883345 |U_11397238|2020-04-20T15:39:4

# Gộp các bảng 

In [22]:
df_reply_agg = df_reply.groupBy('user_id').agg(
    F.count('id').alias('total_replies')
)

df_reply_agg.show()

+----------+-------------+
|   user_id|total_replies|
+----------+-------------+
| U_4647271|            1|
|U_11362449|            7|
|U_13344033|            3|
| U_3406961|            1|
|U_12090588|            2|
|U_11886354|            3|
| U_7138285|            1|
|U_14200158|            6|
|U_11952727|            2|
|U_12716514|            1|
| U_4512141|            1|
|U_11250320|            5|
| U_5078070|            1|
| U_4642459|            8|
|U_12933674|            1|
|U_12106887|            1|
|U_13927517|            2|
|U_13376037|            1|
|  U_648370|            1|
|U_12571954|            1|
+----------+-------------+
only showing top 20 rows



In [23]:
df_master = df_user_course.join(
    df_user_video_final, 
    on=['user_id'],
    how='left'
)

In [24]:
df_master.filter(F.col("video_id").isNotNull()).show()

25/10/16 04:19:45 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block broadcast_69 in memory.
25/10/16 04:19:45 WARN MemoryStore: Not enough space to cache broadcast_69 in memory! (computed 384.0 B so far)
25/10/16 04:19:45 WARN BlockManager: Persisting block broadcast_69 to disk instead.


+---------+---------+--------------------+---------+----------------+----------+------------+------------------+----------------+
|  user_id|course_id|         enroll_time| video_id|total_watch_time|mean_speed|video_length|completion_percent|engagement_level|
+---------+---------+--------------------+---------+----------------+----------+------------+------------------+----------------+
|U_1000290| C_697293|2020-09-25T19:03:...|V_6280777|           672.0|       1.0|    1174.279|            57.227|      Trung bình|
|U_1000290| C_697293|2020-09-25T19:03:...|V_6280707|           362.0|       1.0|      859.22|            42.131|      Trung bình|
|U_1000290| C_697293|2020-09-25T19:03:...|V_6280711|           335.0|       1.0|      792.33|             42.28|      Trung bình|
|U_1000290| C_697293|2020-09-25T19:03:...|V_6280776|           942.0|       1.0|      906.21|             100.0|         Rất cao|
|U_1000454|C_2199449|2020-10-12T13:24:...|V_7396061|           450.0|       1.0|     462.1

In [25]:
df_final = df_master.join(
    df_reply_agg,
    on=['user_id'],
    how='left'
)

In [26]:
print(df_final.columns)

['user_id', 'course_id', 'enroll_time', 'video_id', 'total_watch_time', 'mean_speed', 'video_length', 'completion_percent', 'engagement_level', 'total_replies']


# Xem Schema và số lượng bản ghi

# Lưu kết quả

In [27]:
# df_final.write.mode('overwrite').parquet('kaggle/working/merge_r_uv_uc.parquet')

In [28]:
# import shutil

# src_dir = '/kaggle/working/kaggle/working/merge_r_uv_uc.parquet'
# dst_zip = '/kaggle/working/merge_r_uv_uc_zip'

# shutil.make_archive(dst_zip, 'zip', src_dir)

# print("✅ Đã nén thành công:", dst_zip + ".zip")
